In [1]:
from keras import layers
from keras import models

/home/buddy/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
img_height = 224
img_width = 224
img_channels = 3

cardinality = 32

In [3]:
def residual_network(x):
    def add_common_layers(y):
        y = layers.BatchNormalization()(y)
        y = layers.LeakyReLU()(y)
        
        return y

    def grouped_convolution(y, nb_channels, _strides):
        # when carilandlity ==1 thiis just standard convolution
        if cardinality == 1:
            return layer.Conv2D(nb_channels, kernel_size=(3,3), strides=_strides, padding='same')(y)

        assert not nb_channels % cardinality
        _d = nb_channels // cardinality

        #in a grouped convolution layer, input and output channels are divided into cardinality groups,
        #and convolutions are separately performed within each group
        groups = []
        for j in range(cardinality):
            group = layers.Lambda(lambda z: z[:,:,:, j * _d:j *_d + _d])(y)
            group.append(layers.Conv2D(_d, kernel_size(3,3), strides=_strides, padding='same')(group))

        #the grouped convolutional layer concatenates then as the outputs of the layer
        y = layers.concatenate(groups)

        return y

    def residual_block(y, nb_channels_in, nb_channels_out, _strides=(1,1), _project_shortcut=False):
        shortcut = y
        
        # we modify the residual building block as bottleneck design to make the network more conomical
        y = layers.Conv2D(nb_channels_in, kernel_size=(1,1),strides=(1,1),padding='same')(y)
        y = add_common_layers(y)
        
        # ResNeXt (identical to ResNet when cardinality == 1)
        y = grouped_convolution(y, nb_channels_in, _strides=_strides)
        y = add_common_layers(y)
        
        y = layers.Conv2D(nb_channels_out, kernel_size=(1,1), padding='same')(y)
        #batch normalization is employed after aggregating the transformations and before adding to the shortcut
        y = layers.BatchNormalization()(y)
        
        #identiti shortchuts used directly when the input and output are of the same dimensions
        if _project_shortcut or _strides != (1,1):
            #when the dimensions increase projection shortcut is used to match dimensions (done by 1x1 convolutions)
            #when the shortcuts go across feature maps of two sizes they are performed with a stride of 2
            shortcut = layers.Conv2D(nb_channels_out, kernel_size=(1,1),strides=_strides, padding='same')(shortcut)
            shortcut = layers.BatchNormalization()(shortcut)
        
        y = layers.add([shortcut, y])
        
        #relu is performed right agter each batch normalizaiton,
        #except for the output of the block where relu is peromred after the adding to the shortcut
        y = layers.LeakyReLU()(y)
        
        return y
        
    # conv1
    x = layers.Conv2D(64, kernel_size=(7,7), strides=(2,2),padding='same')(x)
    x = add_common_layers(x)

    # conv2
    x = layers.MaxPool2D(pool_size(3,3), strides=(2,2), padding='same')(x)
    for i in range(3)
        project_shortcut = True if i == 0 else False
        x = residual_block(x, 128, 256, _project_shortcut=project_shortcut)

    # conv3
    for i in range(4)
        #down-sampling is performed by conv3_1, conv4_1 and conv 5_1 with a stride of 2
        strides = (2,2) if == 0 else (1,1)
        x = residual_block(x, 256, 512, _strides)

    # conv4
    for i in range(6):
        strides = (2,2) if i == 0 else (1,1)
        x = residual_block(x,512,1024, _strides)

    # conv5
    for i in range(3)
        strides = (2,2) if i == 0 else (1,1)
        x = residual_block(x, 1024, 2048, _strides=strides)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1)(x)

    return x

SyntaxError: invalid syntax (<ipython-input-3-22c8bf8f3209>, line 64)

In [4]:
image_tensor = layers.Input(shape=(img_height, img_width, img_channels))
network_output = resiidual_network(image_tensor)

model = models.Model(inputs=[image_tensor], outputs=[network_output])
print(model.summary())

NameError: name 'resiidual_network' is not defined